In [2]:
#import pandas
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import re as re


In [3]:

#raw data
Q22016 = 'Raw Data/San_Francisco_Development_Pipeline_2016_Quarter_2.csv'
Q12016 = 'Raw Data/San_Francisco_Development_Pipeline_2016_Quarter_1.csv'
Q42015 = 'Raw Data/San_Francisco_Development_Pipeline_2015_Quarter_4.csv'
Q32015 = 'Raw Data/San_Francisco_Development_Pipeline_2015_Quarter_3.csv'
Q22015 = 'Raw Data/San_Francisco_Development_Pipeline_2015_Quarter_2.csv'
Q12015 = 'Raw Data/San_Francisco_Development_Pipeline_2015_Quarter_1.csv'
Q42014 = 'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_4.csv'
Q32014 = 'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv'
Q22014 = 'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_2.csv'
Q12014 = 'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv'
Q42013 = 'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv'
Q32013 = 'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_3.csv'
Q22013 = 'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_2.csv'
Q12013 = 'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_1.csv'
Q42012 = 'Raw Data/San_Francisco_Development_Pipeline_2012_Quarter_4.csv'


In [4]:
##############################
##Data Cleaning before append
##############################

#First, because variable names vary across files, create dictionary as universal source for common variable neames
variables = {'APN': 'lot_number', 'BLKLOT': 'lot_number', 'Block Lot': 'lot_number',
             'block lot': 'lot_number','NAMEADDR': 'address', 'address': 'address', 
             'BESTSTAT': 'status', 'Best Stat': 'status', 'best stat': 'status', 'BESTDATE': 'latest_date',
             'best date': 'latest_date', 'Best Date': 'latest_date', 'UNITS': 'units', 
             'Units': 'units', 'units': 'units', 'UNITSNET': 'net_units', 'Net Added Units': 'net_units',
             'net added units': 'net_units', 'AFF': 'affordable_units', 'affordable_units': 'affordable_units',
             'AFFNET': 'net_affordable_units','AFFORDABLE': 'affordable_units', 
             'AFFORDABLE_NET': 'net_affordable_units','AFFORDABLENET': 'net_affordable_units', 
             'net_affordable_units': 'net_affordable_units',  'ZONING_SIM': 'zone', 'zoning': 'zone',
             'ZONING': 'zone', 'Zoning_Simplified': 'zone', 'Zoning Simplified': 'zone',
             'zoning_simplified': 'zone', 'Location': 'lat_lon', 
            'LOCATION': 'lat_lon', 'geography': 'lat_lon', 'GEOGRAPHY': 'lat_lon', 
            'Geography': 'lat_lon', 'Location 1': 'lat_lon'}

#Create second dictionary for files where ZONING column is for zoning description instead of zoning code
variables2 = {'APN': 'lot_number', 'BLKLOT': 'lot_number', 'Block Lot': 'lot_number',
             'block lot': 'lot_number','NAMEADDR': 'address', 'address': 'address', 
             'BESTSTAT': 'status', 'Best Stat': 'status', 'best stat': 'status', 'BESTDATE': 'latest_date',
             'best date': 'latest_date', 'Best Date': 'latest_date', 'UNITS': 'units', 
             'Units': 'units', 'units': 'units', 'UNITSNET': 'net_units', 'Net Added Units': 'net_units',
             'net added units': 'net_units', 'AFF': 'affordable_units', 'affordable_units': 'affordable_units',
             'AFFNET': 'net_affordable_units','AFFORDABLE': 'affordable_units', 
             'AFFORDABLE_NET': 'net_affordable_units','AFFORDABLENET': 'net_affordable_units', 
             'net_affordable_units': 'net_affordable_units',  'ZONING_SIM': 'zone', 'zoning': 'zone',
             'Zoning_Simplified': 'zone', 'Zoning Simplified': 'zone', 'ZONING_SIMPLIFY': 'zone',
             'zoning_simplified': 'zone', 'Location': 'lat_lon', 
            'LOCATION': 'lat_lon', 'geography': 'lat_lon', 'GEOGRAPHY': 'lat_lon', 
            'Geography': 'lat_lon', 'Location 1': 'lat_lon'}


#list files without affordable housing information
aff_missing_list = ['Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv', 
                   'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv', 
                   'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv', 
                   'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_3.csv', 
                   'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_2.csv', 
                   'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_1.csv',
                   'Raw Data/San_Francisco_Development_Pipeline_2012_Quarter_4.csv']

address_problem_list = ['Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv',
                       'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_2.csv',
                       'Raw Data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv', 
                       'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv', 
                       'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_3.csv',
                       'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_2.csv',
                       'Raw Data/San_Francisco_Development_Pipeline_2013_Quarter_1.csv',
                       'Raw Data/San_Francisco_Development_Pipeline_2012_Quarter_4.csv']

#set variables to keep 
keep_vars = ['lot_number', 'address', 'status', 'latest_date', 
             'units', 'net_units', 'affordable_units', 
             'net_affordable_units', 'zone', 'lat_lon']

def clean(filename):
    """
    Cleans each dataframe for appending
    """
    dataframe = pd.read_csv(filename)
    
    #clean address variable (only some files)
    if filename in address_problem_list:
        dataframe['address'] = dataframe['Location 1'].str.split('(').str[0].str.rstrip('\n')
        
    else:
        pass
    
    #Generate missing affordable housing for files missing that information
    if filename in aff_missing_list:
        dataframe['affordable_units'] = np.nan
        dataframe['net_affordable_units'] = np.nan
        
    else:
        pass
    
    #remane columns in each file (Q12015 has name of zoning descriptions that other files use for zoning code)
    if filename == 'Raw Data/San_Francisco_Development_Pipeline_2016_Quarter_1.csv':
        dataframe.rename(columns=variables, inplace=True)

    else:
        dataframe.rename(columns=variables2, inplace=True)

    #Keep necessary variables (some files missing affordable units information) 
    dataframe2 = dataframe[keep_vars].copy() #set new dataframe to copy so as to not modify original dataframe

    #clean APN Number to isolate number
    dataframe2['lot_number'] = dataframe2['lot_number'].str.strip('APN ')
    
    #clean lat_lon variable
    dataframe2['lat_lon'] = dataframe2['lat_lon'].str.extract('(\(.*\))', expand = True) #clean some lat-lons with zip code
    dataframe2['lat'] = dataframe2['lat_lon'].str.split(',').str[0].str.strip('(')
    dataframe2['lon'] = dataframe2['lat_lon'].str.split(',').str[1].str.strip(')')
    
    return dataframe2


In [5]:
Q22016 = clean(Q22016)
Q12016 = clean(Q12016)
Q42015 = clean(Q42015)
Q32015 = clean(Q32015)
Q22015 = clean(Q22015)
Q12015 = clean(Q12015)
Q42014 = clean(Q42014)
Q32014 = clean(Q32014)
Q22014 = clean(Q22014)
Q12014 = clean(Q12014)
Q42013 = clean(Q42013)
Q32013 = clean(Q32013)
Q22013 = clean(Q22013)
Q12013 = clean(Q12013)
Q42012 = clean(Q42012)

In [6]:
Q22016.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4712008,1 ARDATH COURT (COMMUNITY CENTER),BP FILED,12/17/2015,0,0,0,0,RH-2,"(37.73424319390, -122.38243028700)",37.73424319390,-122.38243028700
1,6978069,1 EDGAR AV,BP FILED,05/08/2015,1,1,0,0,RH-1,"(37.72039399170, -122.45252132300)",37.72039399170,-122.45252132300
2,0837003,1 FRANKLIN ST,CONSTRUCTION,06/16/2016,35,35,5,5,NCT-3,"(37.77442891680, -122.42111019300)",37.77442891680,-122.42111019300
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,04/07/2016,239,239,0,0,UMU,"(37.76920143310, -122.40347298500)",37.76920143310,-122.40347298500
4,6525001,1 HORACE ST,BP FILED,09/24/2014,2,1,0,0,RH-2,"(37.75072284580, -122.41439795200)",37.75072284580,-122.41439795200


In [7]:
Q12016.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4712008,1 ARDATH COURT (COMMUNITY CENTER),BP FILED,12/17/2015,0,0,0,0,RH-2,"(37.7342431939, -122.382430)",37.7342431939,-122.382430
1,6978069,1 EDGAR AV,BP FILED,05/08/2015,1,1,0,0,RH-1,"(37.7203939917, -122.452521)",37.7203939917,-122.452521
2,0837003,1 FRANKLIN ST,CONSTRUCTION,03/28/2016,35,35,5,5,NCT-3,"(37.7744289168, -122.421110)",37.7744289168,-122.421110
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,03/17/2016,560,560,0,0,UMU,"(37.7692014331, -122.403473)",37.7692014331,-122.403473
4,6525001,1 HORACE ST,BP FILED,09/24/2014,2,1,0,0,RH-2,"(37.7507228458, -122.414398)",37.7507228458,-122.414398


In [8]:
Q42015.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4884025,"HUNTERS POINT SHIPYARD, PHASE II",PL APPROVED,10/27/2015,10172,9916,3345,3089,P,"(37.71938264020, -122.38493438200)",37.71938264020,-122.38493438200
1,7308001,"PARKMERCED, PHASE I",BP FILED,01/25/2016,4666,4666,284,284,PM-R,"(37.72050478190, -122.48387500900)",37.72050478190,-122.48387500900
2,5087003,SCHLAGE LOCK,PL APPROVED,07/22/2014,1679,1679,168,168,MUG,"(37.71165296840, -122.40222246300)",37.71165296840,-122.40222246300
3,9900048,PIER 48 / SEAWALL LOT 337,PL FILED,04/23/2013,1500,1500,0,0,M-2,"(37.77570376350, -122.38613511100)",37.77570376350,-122.38613511100
4,4110001,PIER 70,PL FILED,02/10/2015,1100,1100,0,0,M-2,"(37.75975413610, -122.38699351500)",37.75975413610,-122.38699351500


In [9]:
Q32015.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4644002,700 INNES ST,PL FILED,12/09/2014,980,980,0,0,NC-2,"(37.73018740910, -122.37203675100)",37.73018740910,-122.37203675100
1,6310001,HOPE SF SUNNYDALE,PL FILED,04/28/2010,1700,915,1700,915,RM-1,"(37.71308743010, -122.41899015200)",37.71308743010,-122.41899015200
2,3506004,10 SOUTH VAN NESS AV,PL FILED,04/14/2015,767,767,0,0,C-3-G,"(37.77425927670, -122.41948598900)",37.77425927670,-122.41948598900
3,3725093,925 MISSION ST,PL FILED,05/15/2015,688,688,58,58,C-3-S,"(37.78215365440, -122.40660110900)",37.78215365440,-122.40660110900
4,3746001,201 FOLSOM ST,CONSTRUCTION,09/24/2015,669,669,0,0,RC-4,"(37.78868201650, -122.39185135300)",37.78868201650,-122.39185135300


In [10]:
Q22015.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,3725093,925 MISSION ST,PL FILED,05/15/2015,688,688,58,58,C-3-S,"(37.78215365440, -122.40660110900)",37.78215365440,-122.40660110900
1,3746003,201 FOLSOM ST,CONSTRUCTION,07/23/2015,669,669,0,0,RC-4,"(37.78868201650, -122.39185135300)",37.78868201650,-122.39185135300
2,3737012,245 01ST ST,BP FILED,12/23/2014,603,603,0,0,TB DTR,"(37.78790011200, -122.39459403800)",37.78790011200,-122.39459403800
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,04/21/2015,560,560,0,0,UMU,"(37.76920143220, -122.40347298500)",37.76920143220,-122.40347298500
4,3506007,1500-1580 MISSION ST,PL FILED,12/18/2014,550,550,110,110,C-3-G,"(37.77382218830, -122.41841702400)",37.77382218830,-122.41841702400


In [11]:
Q12015.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,3965021,2435-2445 16TH ST,PL FILED,03/25/2015,53,53,6,6,UMU,"(37.76532087420, -122.41111997700)",37.76532087420,-122.41111997700
1,2628032,271 UPPER TR,BP FILED,03/20/2015,2,2,0,0,RH-2,"(37.76223134390, -122.44543184700)",37.76223134390,-122.44543184700
2,1505014,579 39TH AV,BP FILED,07/15/2013,2,2,0,0,RH-2,"(37.77788047140, -122.49995220600)",37.77788047140,-122.49995220600
3,4757047,1167 PALOU AV,BP FILED,10/28/2014,2,1,0,0,RH-1,"(37.72918617920, -122.38299497600)",37.72918617920,-122.38299497600
4,1809028,3945 JUDAH ST,BP FILED,12/31/2014,6,6,0,0,NC-1,"(37.76030615840, -122.50455910000)",37.76030615840,-122.50455910000


In [12]:
Q42014.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4886008,HUNTERS POINT EXPY,PL APPROVED,08/03/2010 12:00:00 AM,10500,10237,3345,3089,P,"(37.711469000000001, -122.381602)",37.711469000000001,-122.381602
1,7303001,PARKMERCED,PL APPROVED,05/25/2011 12:00:00 AM,8898,5677,0,-3221,PM-OS,"(37.720500000000001, -122.47545100000001)",37.720500000000001,-122.47545100000001
2,1939001,TREASURE ISLAND,PL APPROVED,03/15/2011 12:00:00 AM,8619,7800,2050,1800,TI-OS/TI-R/TI-MU/TI-PCI,"(37.823551500000001, -122.370648)",37.823551500000001,-122.370648
3,6310001,1654 SUNNYDALE AVE,PL FILED,04/28/2010 12:00:00 AM,1700,915,1700,915,RM-1,"(37.712432, -122.41759399999999)",37.712432,-122.41759399999999
4,4167004,1 TURNER TR,PL FILED,06/30/2010 12:00:00 AM,1600,994,0,0,RM-2,"(37.757050900000003, -122.39567030000001)",37.757050900000003,-122.39567030000001


In [13]:
Q32014.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,0686040,1656 Post St,BP Filed,23-Sep-14,4.0,4.0,NaN,NaN,NC-2,"(37.785772000000001, -122.428996)",37.785772000000001,-122.428996
1,3722367,151 Third St,CONSTRUCTION,24-Sep-14,0.0,NaN,NaN,NaN,C-3-O,"(37.785935700000003, -122.40076120000001)",37.785935700000003,-122.40076120000001
2,3507042,1400 Mission St,PL FILED,08-Apr-09,165.0,165.0,NaN,NaN,C-3-G,"(37.775286399999999, -122.4159598)",37.775286399999999,-122.4159598
3,0331001,231 Ellis St,PL Filed,28-Feb-13,7.0,4.0,NaN,NaN,RC-4,"(37.785198800000003, -122.41015710000001)",37.785198800000003,-122.41015710000001
4,1533017,473 11th Av,BP Filed,28-May-09,2.0,1.0,NaN,NaN,RH-2,"(37.779254999999999, -122.46985100000001)",37.779254999999999,-122.46985100000001


In [14]:
Q22014.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,3507042,1400 Mission St,PL FILED,04/08/2009 12:00:00 AM,165.0,165.0,165.0,165.0,C-3-G,"(37.7752864, -122.4159598)",37.7752864,-122.4159598
1,0331001,231 Ellis St,PL Filed,02/28/2013 12:00:00 AM,7.0,4.0,NaN,NaN,RC-4,"(37.7851988, -122.4101571)",37.7851988,-122.4101571
2,3722027,151 Third St,CONSTRUCTION,06/17/2014 12:00:00 AM,0.0,NaN,NaN,NaN,C-3-S,"(37.7859357, -122.4007612)",37.7859357,-122.4007612
3,1533017,473 11th Av,BP Filed,05/28/2009 12:00:00 AM,2.0,1.0,NaN,NaN,RH-2,"(37.779255, -122.469851)",37.779255,-122.469851
4,185029,1469 Pacific Av,PL Filed,07/10/2013 12:00:00 AM,9.0,9.0,NaN,NaN,RM-1,"(37.7950609, -122.419425)",37.7950609,-122.419425


In [15]:
Q12014.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,1450008,5400 Geary Bl,PL Approved,25-Apr-13,39.0,39.0,NaN,NaN,NC-3,"(37.781106999999999, -122.477592)",37.781106999999999,-122.477592
1,1453033,346 22nd Av,CONSTRUCTION,24-Jan-14,1.0,1.0,NaN,NaN,RM-1,"(37.781271699999998, -122.481498)",37.781271699999998,-122.481498
2,1453039,322 22nd Av,PL Approved,03-Mar-11,3.0,2.0,NaN,NaN,RM-1,"(37.781714800000003, -122.4815258)",37.781714800000003,-122.4815258
3,1460024,411 28th Av,BP Filed,15-Apr-08,2.0,1.0,NaN,NaN,RH-2,"(37.781573000000002, -122.488299)",37.781573000000002,-122.488299
4,1466009,451 34th Av,PL Approved,16-Jul-09,2.0,1.0,NaN,NaN,RH-2,"(37.780580299999997, -122.49460500000001)",37.780580299999997,-122.49460500000001


In [16]:
Q42013.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,3722027,151 Third St,PL Approved,13-Jul-10,0.0,NaN,NaN,NaN,C-3-S,"(37.785935700000003, -122.40076120000001)",37.785935700000003,-122.40076120000001
1,1533017,473 11th Av,BP Filed,28-May-09,2.0,1.0,NaN,NaN,RH-2,"(37.779254999999999, -122.46985100000001)",37.779254999999999,-122.46985100000001
2,3507042,1400 Mission St,PL FILED,08-Apr-09,165.0,165.0,NaN,NaN,C-3-G,"(37.775286399999999, -122.4159598)",37.775286399999999,-122.4159598
3,0331001,231 Ellis St,PL Filed,28-Feb-13,7.0,4.0,NaN,NaN,RC-4,"(37.785198800000003, -122.41015710000001)",37.785198800000003,-122.41015710000001
4,0192014,835-845 Jackson St,PL Approved,12-Jul-12,0.0,0.0,NaN,NaN,CRNC,"(37.795692600000002, -122.4092984)",37.795692600000002,-122.4092984


In [17]:
Q32013.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,1643051,25 Balboa St,PL Approved,12-Jan-09,2,1.0,NaN,NaN,RM-2,"(37.777337600000003, -122.45917559999999)",37.777337600000003,-122.45917559999999
1,2396036,2416 30th Av,BP Filed,06-Dec-06,2,2.0,NaN,NaN,NC-2,"(37.742164500000001, -122.48743109999999)",37.742164500000001,-122.48743109999999
2,2515001,2800 Sloat Bl,PL Approved,13-Nov-08,55,55.0,NaN,NaN,NC-2,"(37.735854000000003, -122.50473700000001)",37.735854000000003,-122.50473700000001
3,2799013,950 Corbett Av,CONSTRUCTION,13-Mar-07,3,3.0,NaN,NaN,RM-1,"(37.751832, -122.44373)",37.751832,-122.44373
4,1671008M,850 29th Av,PL Approved,10-Jul-08,2,1.0,NaN,NaN,RH-2,"(37.773362400000003, -122.48841609999999)",37.773362400000003,-122.48841609999999


In [18]:
Q22013.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4172053,1179 Tennessee St,BP REINSTATED,11/03/2009 12:00:00 AM,8,8.0,NaN,NaN,NCT-2,"(37.7559927, -122.3890583)",37.7559927,-122.3890583
1,3746001,201 Folsom St,BP ISSUED,05/29/2013 12:00:00 AM,669,669.0,NaN,NaN,RC-4,"(37.789186, -122.391823)",37.789186,-122.391823
2,3615026,2525 Mission St,BP ISSUED,08/26/2009 12:00:00 AM,0,0.0,NaN,NaN,Mission Street,"(37.756587, -122.418501)",37.756587,-122.418501
3,6942039,244 Granada Av,BP ISSUED,10/09/2008 12:00:00 AM,2,1.0,NaN,NaN,RH-2,"(37.722873, -122.45699)",37.722873,-122.45699
4,5618020,152 Elsie St,BP REINSTATED,02/20/2009 12:00:00 AM,1,1.0,NaN,NaN,RH-1,"(37.742132, -122.418345)",37.742132,-122.418345


In [19]:
Q12013.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,0185029,1469 Pacific Av,BP Filed,31-Oct-12,9.0,9.0,NaN,NaN,RM-1,"(37.795180999999999, -122.419434)",37.795180999999999,-122.419434
1,0192014,835-845 Jackson St,PL Approved,12-Jul-12,0.0,0.0,NaN,NaN,CRNC,"(37.795639000000001, -122.40885400000001)",37.795639000000001,-122.40885400000001
2,3507042,1400 Mission St,PL FILED,08-Apr-09,165.0,165.0,NaN,NaN,C-3-G,"(37.775274000000003, -122.415971)",37.775274000000003,-122.415971
3,0221020,34 Pleasant St,CONSTRUCTION,26-Apr-13,3.0,1.0,NaN,NaN,RM-3,"(37.793047000000001, -122.413365)",37.793047000000001,-122.413365
4,0331001,231 Ellis St,PL Filed,28-Feb-13,7.0,4.0,NaN,NaN,RC-4,"(37.785117, -122.410011)",37.785117,-122.410011


In [20]:
Q42012.head()

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,3027A001,207 Los Palmos Dr,CONSTRUCTION,24-Aug-11,3.0,3.0,NaN,NaN,RH-1,"(37.735778000000003, -122.449054)",37.735778000000003,-122.449054
1,3180001,50 Phelan Av,PL Approved,12-Jul-10,71.0,71.0,NaN,NaN,NC-T,"(37.723638000000001, -122.452174)",37.723638000000001,-122.452174
2,3197010,1446 Ocean Av,BP Filed,31-Oct-08,13.0,13.0,NaN,NaN,NC-T,"(37.724356, -122.45771999999999)",37.724356,-122.45771999999999
3,3507041,1401 Market St,CONSTRUCTION,23-Feb-12,719.0,719.0,NaN,NaN,C-3-G,"(37.776411000000003, -122.417545)",37.776411000000003,-122.417545
4,3509027,49 Grace St,BP Filed,30-Jul-08,2.0,1.0,NaN,NaN,SLR,"(37.775087999999997, -122.414618)",37.775087999999997,-122.414618


In [21]:
#Append quarters

quarters = [Q12016, Q42015, Q32015, Q22015, Q12015, Q42014, 
            Q32014, Q22014, Q12014, Q42013, Q32013, Q22013, Q12013,
           Q42012]

full_df = Q22016.append(quarters)




In [22]:
full_df[full_df['lot_number'] == '3911001']

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,04/07/2016,239.0,239.0,0.0,0.0,UMU,"(37.76920143310, -122.40347298500)",37.76920143310,-122.40347298500
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,03/17/2016,560.0,560.0,0.0,0.0,UMU,"(37.7692014331, -122.403473)",37.7692014331,-122.403473
15,3911001,1 HENRY ADAMS ST,CONSTRUCTION,12/23/2015,560.0,560.0,0.0,0.0,UMU,"(37.76920143310, -122.40347298500)",37.76920143310,-122.40347298500
7,3911001,1 HENRY ADAMS ST,CONSTRUCTION,09/22/2015,560.0,560.0,0.0,0.0,UMU,"(37.76920143220, -122.40347298500)",37.76920143220,-122.40347298500
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,04/21/2015,560.0,560.0,0.0,0.0,UMU,"(37.76920143220, -122.40347298500)",37.76920143220,-122.40347298500
495,3911001,1 HENRY ADAMS ST,CONSTRUCTION,04/21/2015,560.0,560.0,0.0,0.0,UMU,"(37.76920143220, -122.40347298500)",37.76920143220,-122.40347298500
10,3911001,1 HENRY ADAMS ST,CONSTRUCTION,03/24/2015 12:00:00 AM,560.0,560.0,0.0,0.0,UMU,"(37.769666000000001, -122.40386599999999)",37.769666000000001,-122.40386599999999
198,3911001,1 Henry Adams St,BP ISSUED,08-Jul-14,560.0,560.0,NaN,NaN,UMU,"(37.769666000000001, -122.40386599999999)",37.769666000000001,-122.40386599999999
239,3911001,1 Henry Adams St,BP APPROVED,03/14/2014 12:00:00 AM,560.0,560.0,NaN,NaN,UMU,"(37.769666, -122.403866)",37.769666,-122.403866
99,3911001,1 Henry Adams St,BP APPROVED,14-Mar-14,560.0,560.0,NaN,NaN,UMU,"(37.769666000000001, -122.40386599999999)",37.769666000000001,-122.40386599999999


In [23]:
full_df.head(31)

,lot_number,address,status,latest_date,units,net_units,affordable_units,net_affordable_units,zone,lat_lon,lat,lon
0,4712008,1 ARDATH COURT (COMMUNITY CENTER),BP FILED,12/17/2015,0.0,0.0,0.0,0.0,RH-2,"(37.73424319390, -122.38243028700)",37.73424319390,-122.38243028700
1,6978069,1 EDGAR AV,BP FILED,05/08/2015,1.0,1.0,0.0,0.0,RH-1,"(37.72039399170, -122.45252132300)",37.72039399170,-122.45252132300
2,0837003,1 FRANKLIN ST,CONSTRUCTION,06/16/2016,35.0,35.0,5.0,5.0,NCT-3,"(37.77442891680, -122.42111019300)",37.77442891680,-122.42111019300
3,3911001,1 HENRY ADAMS ST,CONSTRUCTION,04/07/2016,239.0,239.0,0.0,0.0,UMU,"(37.76920143310, -122.40347298500)",37.76920143310,-122.40347298500
4,6525001,1 HORACE ST,BP FILED,09/24/2014,2.0,1.0,0.0,0.0,RH-2,"(37.75072284580, -122.41439795200)",37.75072284580,-122.41439795200
5,5688040,1 PUTNAM ST,BP FILED,05/12/2016,1.0,-1.0,0.0,0.0,RH-1,"(37.73967022070, -122.41059708400)",37.73967022070,-122.41059708400
6,1084001B,1 STANYAN ST,BP ISSUED,06/15/2016,13.0,13.0,0.0,0.0,NC-3,"(37.78107236200, -122.45652162400)",37.78107236200,-122.45652162400
7,0086012,1 TELEGRAPH HILL BL,BP FILED,04/25/2016,0.0,0.0,0.0,0.0,P,"(37.80227941540, -122.40584549500)",37.80227941540,-122.40584549500
8,3506004,10 SOUTH VAN NESS AV,PL FILED,04/11/2016,855.0,855.0,0.0,0.0,C-3-G,"(37.77425927670, -122.41948599100)",37.77425927670,-122.41948599100
9,6800004,10 TINGLEY ST,BP FILED,07/09/2015,2.0,2.0,0.0,0.0,RH-1,"(37.72856383840, -122.43210261800)",37.72856383840,-122.43210261800


12614